# Step 1: Data and Model Preparation

In [1]:
exp_name = 'exp_053'

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
codebase = '../../'
sys.path.append(codebase)

In [3]:
from src.dataflow.vg_graph_loader_3 import VGGraphLoader
from src.dataflow.graph_batcher_2_rpn import GraphBatcher

In [4]:
from src.tensorflow.models.vsp_model_002 import Model
from src.tensorflow.engine.trainer_3 import Trainer
from src.tensorflow.engine.util import deploy, load_params
from src.util.util import pickle_save, pickle_load
from src.eval.sgg_evaluator_02 import Evaluator


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(1294)
np.random.seed(1295)

In [6]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_2.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [7]:
img_ids, train_idx, test_idx = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'split_stanford.pkl'))
misc = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'misc_stanford_2.pkl'))
img_no_ent = misc['img_no_ent']
img_no_pred = misc['img_no_pred']
image_size = misc['img_size']

In [8]:
train_idx = np.asarray(list(set(train_idx) - set(img_no_ent)))
np.random.shuffle(train_idx)
val_idx = train_idx[:1000]
train_idx = train_idx[1000:]

In [9]:
proposals = pickle_load(os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'coords.pkl'))

In [10]:
batcher = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1236, 
)
batcher.set_subset(train_idx)
batcher.shuffle()

In [11]:
batcher_val = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_val.set_subset(val_idx)

In [12]:
tf.reset_default_graph()

In [12]:
model = Model(   
    dim_att_head_role_ent=[1024, 1024], 
    dim_att_head_role_pred=[1024, 1024],
    role_edge_mode='full_soft',
    stop_grad_role_edge=True,
    
    dim_state_ent=1024,
    dim_state_pred=1024,
    dim_emb_head_ent=[300],
    dim_conf_head_ent=[],
    dim_conf_head_pred=[],
    dim_emb_head_pred=[300],
    dim_init_head_ent=[1024],

    dim_message_send_head_pred2ent=[1024, 1024],
    dim_message_pool_head_pred2ent=[1024, 1024],
    dim_message_receive_head_pred2ent=[1024, 1024],    
    dim_message_send_head_ent2pred=[1024, 1024],
    dim_message_pool_head_ent2pred=[1024, 1024],
    dim_message_receive_head_ent2pred=[1024, 1024],
    
    num_mp_iter=3,     
    num_proposals=None,

    heat_role=1.0,
    heat_emb_mil=1.0,
    null_att_logit=0.0,
        
    num_align_iter=3,
    alignment_order='best',
    default_act_fn_name='leaky_relu',
    optimizer_type_generator=tf.train.AdamOptimizer,
    
    max_num_pred=100,
    dim_init_head_pred=[],
    init_state_type_pred='trainable',
    
    dim_proposal_feat=1536,
    emb_dict_ent=gl.noun_emb_dict,
    emb_dict_pred=gl.pred_emb_dict,    
    #image_size=448,
    #num_channels=3,
    debugging=False
)



Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Use tf.cast instead.


In [13]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5)))
sess = tf.Session()
sess.run(tf.no_op())

# Step 2: Training


In [ ]:
train_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})
val_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})


In [ ]:
trainer = Trainer(
    model, sess, batcher, batcher_val, train_evaluator, val_evaluator,
    logdir=os.path.join(codebase, 'log', 'tfsummary', exp_name), 
    modeldir=os.path.join(codebase, 'checkpoint', exp_name),
)

In [ ]:
trainer.train (
    max_steps=20000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=10000, 
    hyperparams = {
        'learning_rate_generator': 1e-5 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


../../src/eval/graph_eval_3.py:95: RuntimeWarning: invalid value encountered in true_divide
  per_image_pred_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))
../../src/eval/graph_eval_3.py:219: RuntimeWarning: invalid value encountered in true_divide
  per_image_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))


Step 1000: loss = 3.572000026702881 (2621.978 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.217, 'PerImagePredMR': 0.978, 'PerImageOneHopMP_at100_iou_0.5': 0.002, 'PerImageOneHopMR_at100_iou_0.5': 0.028, 'PerImageOneHopMP_at50_iou_0.5': 0.003, 'PerImageOneHopMR_at50_iou_0.5': 0.023}
Validation Results: {'PerImagePredMP': 0.215, 'PerImagePredMR': 0.97, 'PerImageOneHopMP_at100_iou_0.5': 0.002, 'PerImageOneHopMR_at100_iou_0.5': 0.03, 'PerImageOneHopMP_at50_iou_0.5': 0.004, 'PerImageOneHopMR_at50_iou_0.5': 0.027}
Step 2000: loss = 3.3929998874664307 (5140.357 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.262, 'PerImagePredMR': 0.962, 'PerImageOneHopMP_at100_iou_0.5': 0.003, 'PerImageOneHopMR_at100_iou_0.5': 0.033, 'PerImageOneHopMP_at50_iou_0.5': 0.005, 'PerImageOneHopMR_at50_iou_0.5': 0.028}
Validation Results: {'PerImagePredMP': 0.249, 'PerImagePredMR': 0.949, 'PerImageOneHopMP_at100_iou_0.5': 0.002, 'PerImageOneHopMR_at100_iou_0.5': 0.03, 'PerImageOneHopMP_

In [ ]:
trainer.train (
    max_steps=30000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-6 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


In [28]:
trainer.train (
    max_steps=10000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-7 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 44000: loss = 1.6820000410079956 (2402.352 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.325, 'PerImagePredMR': 0.965, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.053, 'PerImageOneHopMP_at50_iou_0.5': 0.007, 'PerImageOneHopMR_at50_iou_0.5': 0.045}
Validation Results: {'PerImagePredMP': 0.307, 'PerImagePredMR': 0.953, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.059, 'PerImageOneHopMP_at50_iou_0.5': 0.007, 'PerImageOneHopMR_at50_iou_0.5': 0.051}
Step 45000: loss = 2.1549999713897705 (4778.309 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.317, 'PerImagePredMR': 0.969, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.052, 'PerImageOneHopMP_at50_iou_0.5': 0.006, 'PerImageOneHopMR_at50_iou_0.5': 0.044}
Validation Results: {'PerImagePredMP': 0.305, 'PerImagePredMR': 0.952, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.058, 'PerImageOne

KeyboardInterrupt: 

In [ ]:
trainer.save_model('final_1')

# Step 3: Evaluation

In [14]:
load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-48000'))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../../checkpoint/exp_053/ckpt-48000


In [15]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_with_duplicates.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [16]:
test_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})


In [17]:
test_idx = np.asarray(list(set(test_idx) - set(img_no_ent)))

In [18]:
batcher_test = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_test.set_subset(test_idx)

In [19]:
#test_evaluator.reset()
deploy(sess, model, batcher_test, test_evaluator, model.eval_ops,
    hyperparams={
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
    ignore_errors=False,
    debug_mode=False,
    verbose_rate=100
)


Deploying
Batch 1/997 done in 1 seconds. Spent time for read=0.11 (avg 0.11), processing=1.11 (avg 1.11), write=0.00 (avg 0.00) seconds.
Batch 101/997 done in 8 seconds. Spent time for read=0.01 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 201/997 done in 15 seconds. Spent time for read=0.01 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 301/997 done in 22 seconds. Spent time for read=0.02 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 401/997 done in 30 seconds. Spent time for read=0.00 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 501/997 done in 37 seconds. Spent time for read=0.01 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 601/997 done in 44 seconds. Spent time for read=0.01 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 701/997 done in 52 seconds. Spent time for read=0.01 (avg 0.01), processing=0.

In [35]:
test_results = test_evaluator.evaluate()
test_results

{'PerImagePredMP': 0.31596530304931614,
 'PerImagePredMR': 0.9638242496028098,
 'PerImageOneHopMP_at100_iou_0.5': 0.0038787802735600454,
 'PerImageOneHopMR_at100_iou_0.5': 0.053772982299779296,
 'PerImageOneHopMP_at50_iou_0.5': 0.0064173672982808395,
 'PerImageOneHopMR_at50_iou_0.5': 0.04664551777924036}